In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
def preprocess_image(image_path, target_size=(256, 256)):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, target_size)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=0)
    img_pil = Image.fromarray((img[0, 0] * 255).astype(np.uint8), mode='L')
    return img_pil

In [3]:
def get_name_and_class():
    directory_path = os.getcwd()
    files = os.listdir(directory_path + '/dataset')
    image_files = sorted([file for file in files if file.lower().endswith(('.jpeg'))], key=lambda x: int(x.split('.')[0]))
    full_paths = [os.path.join(directory_path + '/dataset/', file) for file in image_files]

    classes = []
    classes.extend(["chavoshi"] * 20)
    classes.extend(["shajarian"] * 23)
    classes.extend(["khaliq"] * 21)
    classes.extend(["radan"] * 25)
    classes.extend(["bayati"] * 21)
    classes.extend(["kianafshar"] * 25)
    classes.extend(["alidoosti"] * 27)
    classes.extend(["qaforian"] * 25)
    classes.extend(["razavian"] * 20)
    classes.extend(["daei"] * 27)
    classes.extend(["attaran"] * 45)
    classes.extend(["beiranvand"] * 32)
    classes.extend(["dolatshahi"] * 24)
    classes.extend(["esfahani"] * 25)
    classes.extend(["hoceini"] * 20)
 
    return full_paths,classes

In [4]:
image_paths,labels = get_name_and_class()
X_train, X_temp, y_train, y_temp = train_test_split(image_paths, labels, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [5]:
class_to_label = {
    "chavoshi": 0,
    "shajarian": 1,
    "khaliq": 2,
    "radan": 3,
    "bayati": 4,
    "kianafshar": 5,
    "alidoosti": 6,
    "qaforian": 7,
    "razavian": 8,
    "daei": 9,
    "attaran": 10,
    "beiranvand": 11,
    "dolatshahi": 12,
    "esfahani": 13,
    "hoceini": 14,
}

In [6]:
class FaceDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = [class_to_label[label] for label in labels]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = preprocess_image(self.image_paths[idx])
        label = self.labels[idx]
        label = torch.tensor(label, dtype=torch.long) 
        if self.transform:
            image = self.transform(image)
        return image, label

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = FaceDataset(X_train, y_train, transform=transform)
val_dataset = FaceDataset(X_val, y_val, transform=transform)
test_dataset = FaceDataset(X_test, y_test, transform=transform)

In [8]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
class FaceRecognitionModel(nn.Module):
    def __init__(self, num_classes):
        super(FaceRecognitionModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 64 * 64, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x))
        x = self.pool(x)
        x = F.leaky_relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 64 * 64)
        x = F.leaky_relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [10]:
model = FaceRecognitionModel(num_classes=len(set(labels)))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
epochs = 10  
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        labels = torch.tensor(labels, dtype=torch.long)  
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            labels = torch.tensor(labels, dtype=torch.long)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_accuracy = correct / total
        average_val_loss = val_loss / len(val_loader)
        print(f"Epoch {epoch + 1}/{epochs} | Validation Loss: {average_val_loss:.4f} | Validation Accuracy: {val_accuracy * 100:.2f}%")

C:\Users\Roham\AppData\Local\Temp\ipykernel_2992\778674191.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)
C:\Users\Roham\AppData\Local\Temp\ipykernel_2992\778674191.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)


Epoch 1/10 | Validation Loss: 3.1384 | Validation Accuracy: 6.58%
Epoch 2/10 | Validation Loss: 2.7253 | Validation Accuracy: 9.21%
Epoch 3/10 | Validation Loss: 2.7057 | Validation Accuracy: 3.95%
Epoch 4/10 | Validation Loss: 2.6883 | Validation Accuracy: 13.16%
Epoch 5/10 | Validation Loss: 2.7000 | Validation Accuracy: 19.74%
Epoch 6/10 | Validation Loss: 2.7319 | Validation Accuracy: 11.84%
Epoch 7/10 | Validation Loss: 2.8547 | Validation Accuracy: 14.47%
Epoch 8/10 | Validation Loss: 2.9872 | Validation Accuracy: 17.11%
Epoch 9/10 | Validation Loss: 2.8624 | Validation Accuracy: 17.11%
Epoch 10/10 | Validation Loss: 3.1497 | Validation Accuracy: 15.79%


In [12]:
torch.save(model.state_dict(), "face_recognition_model.pth")

In [13]:
model.load_state_dict(torch.load("face_recognition_model.pth"))
model.eval()

FaceRecognitionModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=262144, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=15, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [14]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 17.11%
